### Arimoto-Blahut Algorithm for Capacity

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [82]:
def p_exp(e,x):
    """
    Practical exponentiation
    returns 1 if given 0 to the power 0
    """
    if e == 0 and x == 0:
        return 1
    else:
        return e**x

In [83]:
def entropy_1(px):
    """
    Entropy count of one variable vector
    Input: probability
    Output: entropy in nats
    """
    Hx = []
    for i in range(len(px)):
        if px[i] == 0:
            Hx.append(0)
        else:
            Hx.append(-px[i]*np.log(px[i]))
    return np.sum(Hx)

In [84]:
def entropy_2(pxy):
    """
    Entropy count of two variable matrix
    Input: joint probability
    Output: joint entropy in nats
    """
    Hxy = []
    for i in range(len(pxy)):
        for j in range(len(pxy[0])):
            if pxy[i][j] == 0:
                Hxy.append(0)
            else:
                Hxy.append(-pxy[i][j]*np.log(pxy[i][j]))
    return np.sum(Hxy)

In [85]:
def capacity(pxy, entropy_2, entropy_1):
    """
    Capacity count of two variables
    Input: joint distribution
    Output: mutual information in bits
    """
    Hxy = entropy_2(pxy)
    Hy = entropy_1(np.sum(pxy, axis=0))
    Hx = entropy_1(np.sum(pxy, axis=1))
    return (Hx+Hy-Hxy)/np.log(2)

In [86]:
def gen_pxgy(pygx, px, sx, sy):
    """
    Generation of optimum px-given-y
    Input: channel (pygx) and input distribution (assumed as constant)
    Output: optimum px-given-y
    """
    pxgy = np.zeros((sy,sx))
    for i in range(sy):
        mx = 0
        for j in range(sx):
            mx += px[j]*pygx[j][i]
        for j in range(sx):
            pxgy[i][j] = px[j]*pygx[j][i]/mx
    return pxgy

In [109]:
def gen_npx(pygx, pxgy, sx, sy, p_exp):
    """
    Generation of optimum input distribution
    Input: channel (pygx) and px-given-y
    Output: optimum px
    """
    npx = []
    for i in range(sx):
        tx = 1
        for j in range(sy):
            tx = tx * p_exp(pxgy[j][i],pygx[i][j])
        npx.append(tx)
    npx = npx/np.sum(npx)
    return npx

In [110]:
def gen_pxy(pygx, px, sx, sy):
    """
    Generation of joint probability
    Input: channel and input distribution
    Output: joint probability pxy
    """
    pxy = np.zeros((sx,sy))
    for i in range(sx):
        for j in range(sy):
            pxy[i][j] = pygx[i][j]*px[i]
    return pxy

In [111]:
def AB_Capacity(pygx, capacity, entropy_1, entropy_2, gen_pxgy, gen_npx, gen_pxy, p_exp):
    """
    Arimoto-Blahut algorithm for searching capacity of given Channel Pygx
    Uses uniform distribution as first guess
    Stops when input distribution is converged
    Input: channel (pygx)
    Output: optimum input distribution and capacity of channel
    """
    sx = len(pygx)
    sy = len(pygx[0])
    px = np.zeros(sx)
    npx = []
    for i in range(sx):
        npx.append(1/sx)
    while np.linalg.norm(npx-px) > 10**-6:
        px = npx
        pxgy = gen_pxgy(pygx, px, sx, sy)
        npx = gen_npx(pygx, pxgy, sx, sy, p_exp)
    px = npx
    print("Optimum channel usage is:", px)
    pxy = gen_pxy(pygx, px, sx, sy)
    cap = capacity(pxy, entropy_2, entropy_1)
    print("Maximum capacity is:", cap)
    return cap

In [112]:
# pygx = np.array([[0.75, 0.25, 0],
#                 [0, 1, 0],
#                 [0, 0.5, 0.5]])
pygx = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
                [5, 2, 8, 1, 3, 6, 4, 7, 9],
                [5, 8, 2, 9, 7, 4, 6, 3, 1]])/45
Capacity = AB_Capacity(pygx, capacity, entropy_1, entropy_2, gen_pxgy, gen_npx, gen_pxy, p_exp)

Optimum channel usage is: [0.00194499 0.49857312 0.49948189]
Maximum capacity is: 0.21262939018889393
